In [1]:
import pandas as pd
import os
from time import time
from IPython.display import display # Allows the use of display() for DataFrames
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# Import supplementary visualization code visuals.py
#import visuals as vs

# Pretty display for notebooks
%matplotlib inline

 


### Setting files and directories

In [2]:
path_to_data= "/home/ubuntu/udacity/CodeGladiator/invesco/data"

transaction_file = "Code-Gladiators-Transaction.csv"

####### adding new transaction fiel which have grouping of all the transactions
#grouped_transaction_amounts = "grouped_transaction_amounts.csv"
#investment_exp_file = "Code-Gladiators-InvestmentExperience.csv"
investment_exp_file = "imputed_investment_exp.csv"
investment_segment = "investment_vehicle_segment.csv"
aum_file = "Code-Gladiators-AUM.csv"
#activity_file = "Code-Gladiators-Activity.csv"
activity_file = "grouped_processed_activity.csv"
test_file = "test_data.csv"



### reading csv files into pandas dataframe

In [5]:
transaction_df = pd.read_csv(os.path.join(path_to_data, transaction_file))
#grouped_transaction_amounts_df = pd.read_csv(os.path.join(path_to_data, grouped_transaction_amounts))
investment_exp_df = pd.read_csv(os.path.join(path_to_data, investment_exp_file))
investment_segment_df = pd.read_csv(os.path.join(path_to_data, investment_segment))
aum_df = pd.read_csv(os.path.join(path_to_data, aum_file))
activity_df = pd.read_csv(os.path.join(path_to_data, activity_file))
test_df = pd.read_csv(os.path.join(path_to_data,test_file))

In [6]:
# TODO: Import two metrics from sklearn - fbeta_score and accuracy_score
from sklearn.metrics import fbeta_score, accuracy_score
# time.time() return time in seconds since the Epoch
from time import time

beta = 0.5

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size'
    X_train = X_train[:sample_size]
    y_train = y_train[:sample_size]
    
    start = time() # Get start time
    learner.fit(X_train, y_train)
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = end - start
        
    # TODO: Get the predictions on the test set,
    #       then get predictions on the first 300 training samples
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train)
    end = time() # Get end time
    
    # TODO: Calculate the total prediction time
    results['pred_time'] = end-start
            
    # TODO: Compute accuracy on the first 300 training samples
    results['acc_train'] = accuracy_score(y_train, predictions_train)
        
    # TODO: Compute accuracy on test set
    results['acc_test'] = accuracy_score(y_test, predictions_test)
    
    # TODO: Compute F-score on the the first 300 training samples
    results['f_train'] = fbeta_score(y_train, predictions_train, beta=beta)
        
   # TODO: Compute F-score on the test set
    results['f_test'] = fbeta_score(y_test, predictions_test, beta=beta)
       
    # Success
    print ("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results

In [3]:
#column_list= ['AUM_investor_log','Counts_investor','Shares_investor_log','AUM_advisor_log','Shares_advisor_log','Rating','1 Yr % Rank','3 Yr % Rank','1 Yr Return','3 Yr Return','Net Flows','Morningstar_Category_Rating','Transaction_Type']
see_activity= False

column_list= [
    'Transaction_Type',
#    'Shares_investor',
#    'AUM_investor',
     'Counts_investor',
     'Counts_advisor',
#	'Shares_advisor',
#	'AUM_advisor',
#	'Morningstar Category',
#	'Investment',
	'Rating',
	'1 Yr % Rank',
	'3 Yr % Rank',
#	'5 Yr % Rank',
#	'10 Yr % Rank',
	'1 Yr Return',
	'3 Yr Return',
#	'5 Yr Return',
#	'10 Yr Return',
	'1 Yr Excess Return vs Primary Ix',
	'3 Yr Excess Return vs Primary Ix',
#	'5 Yr Excess Return vs Primary Ix',
#	'10 Yr Excess Return vs Primary Ix',
	'1 Yr Excess Return vs Category Ix',
	'3 Yr Excess Return vs Category Ix',
#	'5 Yr Excess Return vs Category Ix',
#	'10 Yr Excess Return vs Category Ix',
	'Net Flows',
	'Morningstar_Category_Rating',
#	'investment_vehicle_segment',
	'AUM_investor_log',
	'Shares_investor_log',
	'AUM_advisor_log',
	'Shares_advisor_log',
	]

if(see_activity):
    column_list.extend(list(activity_df.columns[pd.Series(activity_df.columns).str.startswith('Activity')]))


display(column_list)


['Transaction_Type',
 'Counts_investor',
 'Counts_advisor',
 'Rating',
 '1 Yr % Rank',
 '3 Yr % Rank',
 '1 Yr Return',
 '3 Yr Return',
 '1 Yr Excess Return vs Primary Ix',
 '3 Yr Excess Return vs Primary Ix',
 '1 Yr Excess Return vs Category Ix',
 '3 Yr Excess Return vs Category Ix',
 'Net Flows',
 'Morningstar_Category_Rating',
 'AUM_investor_log',
 'Shares_investor_log',
 'AUM_advisor_log',
 'Shares_advisor_log']

### processing data

Grouping data by unique advisor id and month. Final output will store sums of the assets under managements and shares for each advisor in particular month

In [35]:
transaction_df = pd.read_csv(os.path.join(path_to_data, transaction_file))
investment_exp_df = pd.read_csv(os.path.join(path_to_data, investment_exp_file))
#grouped_transaction_amounts_df = pd.read_csv(os.path.join(path_to_data, grouped_transaction_amounts))
investment_segment_df = pd.read_csv(os.path.join(path_to_data, investment_segment))
aum_df = pd.read_csv(os.path.join(path_to_data, aum_file))
activity_df = pd.read_csv(os.path.join(path_to_data, activity_file))
test_df = pd.read_csv(os.path.join(path_to_data,test_file))




grouped_advisor_aum_df = aum_df.groupby(['Unique_Advisor_Id','Month']).agg({'AUM': 'mean','Shares':'mean','Unique_Investment_Id':'count'}).reset_index().rename(columns={'Unique_Investment_Id':'Counts'})
grouped_investment_aum_df = aum_df.groupby(['Unique_Investment_Id','Month']).agg({'AUM': 'mean','Shares':'mean','Unique_Advisor_Id':'count'}).reset_index().rename(columns={'Unique_Advisor_Id':'Counts'})

grouped_investment_aum_df['Year'],grouped_investment_aum_df['Month']=grouped_investment_aum_df['Month'].str.split(' /', 1).str
grouped_advisor_aum_df['Year'],grouped_advisor_aum_df['Month']=grouped_advisor_aum_df['Month'].str.split(' /', 1).str

transaction_df['Year'],transaction_df['Month']=transaction_df['Month'].str.split(' /', 1).str
investment_exp_df['Year'],investment_exp_df['Month']=investment_exp_df['Month'].str.split(' /', 1).str
aum_df['Year'],aum_df['Month']=aum_df['Month'].str.split(' /', 1).str
activity_df['Year'],activity_df['Month']=activity_df['Month'].str.split(' /', 1).str
#grouped_transaction_amounts_df['Year'],grouped_transaction_amounts_df['Month'] =grouped_transaction_amounts_df['Month'].str.split(' /', 1).str

grouped_investment_aum_df['Mapping_Month']= grouped_investment_aum_df['Month'].astype(int).apply(lambda x : x+1)
grouped_advisor_aum_df['Mapping_Month']= grouped_advisor_aum_df['Month'].astype(int).apply(lambda x : x+1)
investment_exp_df['Mapping_Month']= investment_exp_df['Month'].astype(int).apply(lambda x : x+1)
aum_df['Mapping_Month']= aum_df['Month'].astype(int).apply(lambda x : x+1)
activity_df['Mapping_Month']= activity_df['Month'].astype(int).apply(lambda x : x+1)
#grouped_transaction_amounts_df['Mapping_Month']= grouped_transaction_amounts_df['Month'].astype(int).apply(lambda x : x+1)


transaction_df['Month'] =  transaction_df['Month'].astype(int).apply(lambda x : x+0)

final_transaction = pd.merge(transaction_df, grouped_investment_aum_df, left_on=["Month","Unique_Investment_Id"],right_on=["Mapping_Month","Unique_Investment_Id"], how="left") 
final_transaction = final_transaction.rename(columns={'Month_x': 'Month', 'AUM': 'AUM_investor','Year_x' : 'Year', 'Counts' : 'Counts_investor', 'Shares': 'Shares_investor','Month_y':'Month_actual'})
final_transaction = final_transaction.drop('Year_y', 1)

test_transaction = pd.merge(test_df, grouped_investment_aum_df[grouped_investment_aum_df["Mapping_Month"]== 13], on="Unique_Investment_Id", how="left")
test_transaction = test_transaction.rename(columns={ 'AUM': 'AUM_investor','Counts' : 'Counts_investor', 'Shares': 'Shares_investor'})

final_transaction = pd.merge(final_transaction, grouped_advisor_aum_df, left_on=["Month","Unique_Advisor_Id"],right_on=["Mapping_Month","Unique_Advisor_Id"], how="left") 
final_transaction = final_transaction.rename(columns={'Month_x': 'Month', 'AUM': 'AUM_advisor','Year_x' : 'Year', 'Counts' : 'Counts_advisor', 'Shares': 'Shares_advisor','Mapping_Month_x': 'Mapping_Month'})
final_transaction = final_transaction.drop(['Year_y','Mapping_Month_y','Month_y'], 1)

test_transaction = pd.merge(test_transaction, grouped_advisor_aum_df[grouped_advisor_aum_df["Mapping_Month"]== 13], on="Unique_Advisor_Id", how="left")
test_transaction = test_transaction.rename(columns={ 'AUM': 'AUM_advisor','Counts' : 'Counts_advisor', 'Shares': 'Shares_advisor','Month_x':'Month','Mapping_Month_x':'Mapping_Month'})
test_transaction = test_transaction.drop(['Year_x','Year_y','Mapping_Month_y','Month_y','Mapping_Month'], 1)

investment_exp_df['investment_vehicle_segment']= investment_segment_df['investment_vehicle_segment']
investment_exp_df= investment_exp_df[investment_exp_df['Year']=='2016']
activity_df = activity_df[activity_df['Year']=='2016']

final_transaction_with_exp = pd.merge(final_transaction, investment_exp_df, left_on=["Month","Unique_Investment_Id"],right_on=["Mapping_Month","Unique_Investment_Id"], how="left") 
final_transaction_with_exp = final_transaction_with_exp.rename(columns={'Month_x': 'Month', 'AUM': 'AUM_advisor','Year_x' : 'Year', 'Mapping_Month_x':'Mapping_Month'})
final_transaction_with_exp = final_transaction_with_exp.drop(['Year_y','Mapping_Month_y','Month_y'], 1)
#final_transaction_with_exp = pd.merge(final_transaction_with_exp, grouped_transaction_amounts_df, left_on=["Month","Unique_Investment_Id"],right_on=["Mapping_Month","Unique_Investment_Id"], how="left") 
#final_transaction_with_exp = final_transaction_with_exp.drop(['Year_y','Mapping_Month_y','Month_y'], 1)

test_transaction_with_exp = pd.merge(test_transaction, investment_exp_df[investment_exp_df["Mapping_Month"]== 13 ], on="Unique_Investment_Id", how="left")
#test_transaction_with_exp = pd.merge(test_transaction_with_exp, grouped_transaction_amounts_df[grouped_transaction_amounts_df["Mapping_Month"]== 13 ], on=["Unique_Investment_Id"], how="left")

#display(final_transaction_with_exp.head())
#display(test_transaction_with_exp.head())

##merging with activity
if(see_activity):
    final_transaction_with_exp = pd.merge(final_transaction_with_exp, activity_df, left_on=["Month","Unique_Advisor_Id"],right_on=["Mapping_Month","Unique_Advisor_Id"], how="left") 
    test_transaction_with_exp = pd.merge(test_transaction_with_exp, activity_df[activity_df["Mapping_Month"]== 13 ], on="Unique_Advisor_Id", how="left")


final_transaction_with_exp['AUM_investor_log'] = (np.log(final_transaction_with_exp['AUM_investor'])).where(final_transaction_with_exp['AUM_investor']!=0)
final_transaction_with_exp['Shares_investor_log'] = (np.log(final_transaction_with_exp['Shares_investor'])).where(final_transaction_with_exp['Shares_investor']!=0)

final_transaction_with_exp['AUM_advisor_log'] = (np.log(final_transaction_with_exp['AUM_advisor'])).where(final_transaction_with_exp['AUM_advisor']!=0)
final_transaction_with_exp['Shares_advisor_log'] = (np.log(final_transaction_with_exp['Shares_advisor'])).where(final_transaction_with_exp['Shares_advisor']!=0)


test_transaction_with_exp['AUM_investor_log'] = (np.log(test_transaction_with_exp['AUM_investor'])).where(test_transaction_with_exp['AUM_investor']!=0)
test_transaction_with_exp['Shares_investor_log'] = (np.log(test_transaction_with_exp['Shares_investor'])).where(test_transaction_with_exp['Shares_investor']!=0)

test_transaction_with_exp['AUM_advisor_log'] = (np.log(test_transaction_with_exp['AUM_advisor'])).where(test_transaction_with_exp['AUM_advisor']!=0)
test_transaction_with_exp['Shares_advisor_log'] = (np.log(test_transaction_with_exp['Shares_advisor'])).where(test_transaction_with_exp['Shares_advisor']!=0)

final_transaction_with_exp= final_transaction_with_exp[final_transaction_with_exp.Month !=1]

#final_transaction_with_exp= final_transaction_with_exp.dropna()


required_train_df = final_transaction_with_exp.filter(column_list)
required_train_df['Transaction_Type']= required_train_df.apply(lambda x: 0 if x['Transaction_Type']== 'P' else 1, axis=1)

if(see_activity):
    required_train_df[required_train_df.columns[pd.Series(required_train_df.columns).str.startswith('Activity')]] = required_train_df[required_train_df.columns[pd.Series(required_train_df.columns).str.startswith('Activity')]].fillna(0)

required_test_df = test_transaction_with_exp.filter(column_list)

if(see_activity):
    required_test_df[required_test_df.columns[pd.Series(required_test_df.columns).str.startswith('Activity')]] = required_test_df[required_test_df.columns[pd.Series(required_test_df.columns).str.startswith('Activity')]].fillna(0)
    
required_train_df_meanNa= required_train_df.fillna(required_train_df.median())
required_test_df_meanNa = required_test_df.fillna(required_test_df.median())
required_train_df_na = required_train_df_meanNa[required_train_df.isnull().any(axis=1)]
required_test_df_na = required_test_df_meanNa[required_test_df.isnull().any(axis=1)]

required_train_df_noNa= required_train_df_meanNa[required_train_df.isnull().any(axis=1)== False]
required_test_df_noNa = required_test_df_meanNa[required_test_df.isnull().any(axis=1)== False]

test_df_na = test_df[required_test_df.isnull().any(axis=1)]
test_df_noNa = test_df[required_test_df.isnull().any(axis=1)== False]



# Split the data into features and target label
transaction_type_na = required_train_df_na['Transaction_Type']
transaction_type_noNa = required_train_df_noNa['Transaction_Type']

features_raw_na = required_train_df_na.drop('Transaction_Type', axis = 1)
test_raw_na = required_test_df_na
features_raw_noNa = required_train_df_noNa.drop('Transaction_Type', axis = 1)
test_raw_noNa = required_test_df_noNa


from sklearn.preprocessing import LabelEncoder
var_mod = column_list
#var_mod.remove('Transaction_Type')
le = LabelEncoder()
for i in var_mod:
    if i == 'Transaction_Type' \
    or i == 'Net Flows' \
    or i == 'AUM_investor_log' \
    or i == 'Shares_investor_log' \
    or i == 'AUM_advisor_log' \
    or i == 'Shares_advisor_log':
        continue
    features_raw_na[i] = le.fit_transform(features_raw_na[i])
    test_raw_na[i] = le.fit_transform(test_raw_na[i])
    features_raw_noNa[i] = le.fit_transform(features_raw_noNa[i])
    test_raw_noNa[i] = le.fit_transform(test_raw_noNa[i])
    
###########################################################3


from sklearn.cross_validation import train_test_split

# Split the 'features' and 'transaction_type' data into training and testing sets
X_train_na, X_test_na, y_train_na, y_test_na = train_test_split(features_raw_na, transaction_type_na, test_size = 0.2, random_state = 0)
X_train_noNa, X_test_noNa, y_train_noNa, y_test_noNa = train_test_split(features_raw_noNa, transaction_type_noNa, test_size = 0.2, random_state = 0)


# Show the results of the split
print ("Training na set has {} samples.".format(X_train_na.shape[0]))
print ("Testing na set has {} samples.".format(X_test_na.shape[0]))

print ("Training noNa set has {} samples.".format(X_train_noNa.shape[0]))
print ("Testing noNa set has {} samples.".format(X_test_noNa.shape[0]))

#############################################################################


# TODO: Import the three supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# TODO: Initialize the three models
clf_A_na = LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)
clf_A_noNa = LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)
clf_B = LinearSVC(random_state=101)
clf_C = GaussianNB()
clf_Ada = AdaBoostClassifier()
clf_Grad = GradientBoostingClassifier()
clf_KNN = KNeighborsClassifier()
clf_Dec = DecisionTreeClassifier()
clf_SGD = SGDClassifier()
clf_Ran = RandomForestClassifier(n_estimators = 100)


# TODO: Calculate the number of samples for 1%, 10%, and 100% of the training data
n_train = len(y_train_na)
samples_1 = int(n_train * 0.01)
samples_10 = int(n_train * 0.1)
samples_100 = n_train

# Collect results on the learners
results_na = {}
#for clf in [clf_A, clf_B, clf_C, clf_Ada, clf_Grad,clf_KNN ,clf_Dec, clf_SGD]:
for clf in [clf_A_na]:
    clf_name = clf.__class__.__name__
    results_na[clf_name] = {}
    for i, samples in enumerate([samples_100]):
        results_na[clf_name][i] = \
        train_predict(clf, samples, X_train_na, y_train_na, X_test_na, y_test_na)
        
display(results_na)

n_train = len(y_train_noNa)
samples_1 = int(n_train * 0.01)
samples_10 = int(n_train * 0.1)
samples_100 = n_train
# Collect results on the learners
results_noNa = {}
#for clf in [clf_A, clf_B, clf_C, clf_Ada, clf_Grad,clf_KNN ,clf_Dec, clf_SGD]:
for clf in [clf_A_noNa]:
    clf_name = clf.__class__.__name__
    results_noNa[clf_name] = {}
    for i, samples in enumerate([samples_100]):
        results_noNa[clf_name][i] = \
        train_predict(clf, samples, X_train_noNa, y_train_noNa, X_test_noNa, y_test_noNa)
        
display(results_noNa)


from sklearn.externals import joblib
for best_clf in [clf_A_na,clf_A_noNa]:
    if best_clf == clf_A_na:
        filename = 'logistic_regression_NA_invesco.joblib.pkl'
        test_raw= test_raw_na
    else:
        filename = 'logistic_regression_noNA_invesco.joblib.pkl'
        test_raw= test_raw_noNa

    _ = joblib.dump(best_clf, filename, compress=9)

    clf_loaded = joblib.load(filename)
    print("file loaded")

    pred = clf_loaded.predict(test_raw)
    pred_prob = clf_loaded.predict_proba(test_raw)

    pred_prob = pd.DataFrame(pred_prob[:,1],columns=["Propensity_Score"])
    display(pred_prob[:5])

    pred_df= pd.DataFrame(pred,columns=["Redeem_Status"])

    pred_df=pred_df.replace([0,1],['NO','YES'])
    pred_df.head()

    display(pred_df['Redeem_Status'].value_counts())
    
    if best_clf == clf_A_na:
        result_na = pd.concat([test_df_na.reset_index(drop=True), pred_prob.reset_index(drop=True), pred_df.reset_index(drop=True)],ignore_index=True, axis=1)
        
    else:
        result_noNa = pd.concat([test_df_noNa.reset_index(drop=True), pred_prob.reset_index(drop=True), pred_df.reset_index(drop=True)],ignore_index=True ,axis=1)
    
display(result_na)
display(result_noNa)

/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:101: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:103: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:104: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:108: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:111: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/.local/lib/pytho

Training na set has 19409 samples.
Testing na set has 4853 samples.
Training noNa set has 99782 samples.
Testing noNa set has 24946 samples.


/home/ubuntu/.local/lib/python3.5/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ubuntu/.local/lib/python3.5/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


LogisticRegression trained on 19409 samples.


/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


...

file loaded


,Propensity_Score
0,0.943226
1,0.613030
2,0.266147
3,0.646299
4,0.565217


YES    6911
NO      302
Name: Redeem_Status, dtype: int64

,0,1,2,3
0,1000332,14146,0.835213,YES
1,1000332,18654,0.026404,NO
2,1000332,3460,0.067945,NO
3,1000533,5538,0.156935,NO
4,1003125,3460,0.067945,NO
5,1009195,15354,0.150220,NO
6,1009195,3533,0.001642,NO
7,1009570,18381,0.167886,NO
8,1009570,5841,0.167886,NO
9,1014856,9320,0.000386,NO


,0,1,2,3
0,1000103,14147,0.943226,YES
1,1000103,3534,0.613030,YES
2,1000103,3651,0.266147,NO
3,1000103,7668,0.646299,YES
4,1000103,9339,0.565217,YES
5,1000373,4008,0.876855,YES
6,1000541,4008,0.875005,YES
7,1001726,11540,0.908851,YES
8,1001726,3532,0.904557,YES
9,1001726,3538,0.877992,YES


In [14]:
result_na.shape

(1501, 4)

In [34]:
print(required_test_df_na.shape)
print(required_test_df_meanNa.shape)
print(required_test_df_noNa.shape)

(1501, 17)
(8714, 17)
(7213, 17)


In [36]:
na_index_list= test_df[required_test_df.isnull().any(axis=1)].index.tolist()
noNa_index_list = test_df[required_test_df.isnull().any(axis=1)== False].index.tolist()
result_na = result_na.set_index([na_index_list]) 
result_noNa = result_noNa.set_index([noNa_index_list])



display(result_na.head())
display(result_noNa.head())

,0,1,2,3
5,1000332,14146,0.835213,YES
6,1000332,18654,0.026404,NO
7,1000332,3460,0.067945,NO
9,1000533,5538,0.156935,NO
21,1003125,3460,0.067945,NO


,0,1,2,3
0,1000103,14147,0.943226,YES
1,1000103,3534,0.613030,YES
2,1000103,3651,0.266147,NO
3,1000103,7668,0.646299,YES
4,1000103,9339,0.565217,YES


In [38]:
final_test= pd.concat([result_na,result_noNa]).sort_index()
final_test.head(10)



,0,1,2,3
0,1000103,14147,0.943226,YES
1,1000103,3534,0.613030,YES
2,1000103,3651,0.266147,NO
3,1000103,7668,0.646299,YES
4,1000103,9339,0.565217,YES
5,1000332,14146,0.835213,YES
6,1000332,18654,0.026404,NO
7,1000332,3460,0.067945,NO
8,1000373,4008,0.876855,YES
9,1000533,5538,0.156935,NO


In [39]:
final_test.columns = ['Unique_Advisor_Id','Unique_Investment_Id','Propensity_Score','Redeem_Status']
final_test.head(10)

,Unique_Advisor_Id,Unique_Investment_Id,Propensity_Score,Redeem_Status
0,1000103,14147,0.943226,YES
1,1000103,3534,0.613030,YES
2,1000103,3651,0.266147,NO
3,1000103,7668,0.646299,YES
4,1000103,9339,0.565217,YES
5,1000332,14146,0.835213,YES
6,1000332,18654,0.026404,NO
7,1000332,3460,0.067945,NO
8,1000373,4008,0.876855,YES
9,1000533,5538,0.156935,NO


In [40]:


final_test.to_csv('test_data_v7-sepNAs.csv',index=False)


In [59]:
#result.to_csv('test_data_v5-removednetflows.csv',index=False)